In [2]:
import pandas as pd
import re
import os

import matplotlib.pyplot as plt

from pyspark.sql import SparkSession
from pyspark.sql.functions import unix_timestamp, udf, lit, explode, split, regexp_extract, col, isnan, isnull, desc, when, sum, to_date, desc, regexp_replace, count, to_timestamp
from pyspark.sql.types import IntegerType, TimestampType

In [3]:
#setting visualization options
# https://www.1week4.com/it/machine-learning/udacity-data-engineering-capstone-project/
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  

# modify visualization of the notebook, for easier view
from IPython.core.display import display, HTML
display(HTML("""<style> p { max-width:90% !important; } h1 {font-size:2rem!important } h2 {font-size:1.6rem!important } 
h3 {font-size:1.4rem!important } h4 {font-size:1.3rem!important }h5 {font-size:1.2rem!important }h6 {font-size:1.1rem!important }</style>"""))# Do all imports and installs here


In [4]:
def create_spark_session():
    """
    This function creates a Spark Sesson and includes necessary Jar and adoop packages in the configuration. 
    """
    spark=SparkSession \
    .builder \
    .config("spark.jars.repositories", "https://repos.spark-packages.org/") \
    .config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11") \
    .enableHiveSupport() \
    .getOrCreate()
    return spark

In [5]:
spark = create_spark_session()

In [6]:
df_airport_codes = spark.read.csv('../../../airport-codes_csv.csv', sep=',', inferSchema=True, header=True)

In [9]:
df_airport_codes.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



In [11]:
df_airport_codes.limit(5).toPandas().head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,"-91.254898, 35.6087"


In [15]:
df_airport_codes.count()

55075

In [14]:
# https://stackoverflow.com/questions/44627386/how-to-find-count-of-null-and-nan-values-for-each-column-in-a-pyspark-dataframe
from pyspark.sql.functions import isnan, when, count, col

df_airport_codes.select([count(when(isnull(c), c)).alias(c) for c in df_airport_codes.columns]).show()


+-----+----+----+------------+---------+-----------+----------+------------+--------+---------+----------+-----------+
|ident|type|name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|coordinates|
+-----+----+----+------------+---------+-----------+----------+------------+--------+---------+----------+-----------+
|    0|   0|   0|        7006|        0|          0|         0|        5676|   14045|    45886|     26389|          0|
+-----+----+----+------------+---------+-----------+----------+------------+--------+---------+----------+-----------+



In [15]:
df_airport_codes.where(isnull('local_code')).count()

26389

In [14]:
df_airport_codes.where(isnull('local_code')).select("name").groupBy("name").count().orderBy(desc('count')).show()
# df_I94.df_airport_codes(col('local_code') > col('depDate')).count().orderBy(desc('count')

+--------------------+-----+
|                name|count|
+--------------------+-----+
|Centre Hospitalie...|   85|
| Mukho Port Heliport|   50|
|Cheonmi-ri South ...|   16|
|   Hospital Heliport|   16|
|Fazenda Santa Mar...|   13|
|Fazenda SÃ£o JoÃ£...|   13|
|Fazenda Bela Vist...|   10|
|Fazenda SÃ£o Fran...|    9|
|Fazenda Santa Rit...|    8|
|Fazenda SÃ£o JosÃ...|    8|
| Santa Maria Airport|    8|
|Fazenda Primavera...|    6|
|   San Pedro Airport|    6|
|   Aeroclube Airport|    6|
|La Esperanza Airport|    6|
|Fazenda Colorado ...|    6|
|Fazenda SÃ£o Seba...|    6|
|Fazenda Santo Ant...|    6|
|Fazenda Santa FÃ©...|    6|
|  Santa Rita Airport|    5|
+--------------------+-----+
only showing top 20 rows

